挂载硬盘

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


配置 github

In [2]:
!mkdir /root/.ssh

In [3]:
!cp -f /content/gdrive/My\ Drive/id_rsa /root/.ssh

In [4]:
!chmod 600 /root/.ssh/id_rsa

In [5]:
!ssh-keyscan github.com >> /root/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-9389d978
# github.com:22 SSH-2.0-babeld-9389d978
# github.com:22 SSH-2.0-babeld-9389d978


In [6]:
!git config --global user.email "287665530@qq.com"

In [ ]:
import os
#os.chdir('/content/gdrive/My Drive/Colab Notebooks/workshop/keras-models')

In [ ]:
!git pull

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 10 (delta 6), reused 10 (delta 6), pack-reused 0
Unpacking objects: 100% (10/10), done.
From github.com:jinxueyu/keras-models
   2d061e2..1b1b391  master     -> origin/master
Updating 2d061e2..1b1b391
Fast-forward
 src/main/py/nlp/segment/lac.py   | 47 +++++++++++++++++++++++++++-------------
 src/main/py/nlp/segment/model.py | 36 +++++++++++++++++++-----------
 src/main/py/nlp/segment/tools.py | 39 +++++++++++++++++++++++++--------
 3 files changed, 85 insertions(+), 37 deletions(-)


In [ ]:
!git add .
!git commit -m 'test from colab'
!git push

In [7]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/workshop/keras-models/src/main/py')

In [8]:
import configparser
import os

import tensorflow as tf

from nlp.corpus.reader import DataProcessor
from nlp.segment.model import build_glove_bi_gru_crf_model, build_glove_bi_gru2_crf_model, MAX_LENGTH
from nlp.segment.seg import SegmentBase
from util import ObjectDict
from nlp.segment.crf import CRFSegment

In [10]:
config = configparser.ConfigParser()
config.read(os.path.join('', "model-args-config.ini"))

data_path = '/content/gdrive/My Drive/data'
model_path = 'nlp/model/keras-models'
corpus_path = 'nlp/corpus/icwb2-data/training/msr_training.utf8'
glove_path = 'nlp/embidding/glove/tencent/Tencent_AILab_ChineseEmbedding_Single.txt'

dataset = DataProcessor()

mode = 'seg'
mode = 'train'

In [16]:
def build_glove_bi_gru2_td_crf_model(args, word_to_id_func):
    vocab_size = args.vocab_size
    num_labels = args.num_labels
    embedding_glove_path = args.glove_path

    model = Sequential()
    embedding_layer = build_glove_layer(embedding_glove_path, vocab_size, word_to_id_func)
    model.add(embedding_layer)

    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(Dropout(DROPOUT_RATE))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(Dropout(DROPOUT_RATE))
    model.add(TimeDistributed(Dense(num_labels, activation='softmax')))

    crf_layer = build_crf_layer(num_labels)
    model.add(crf_layer)
    model.compile('adam', loss=crf_loss, metrics=[crf_layer.get_accuracy])

    # tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)
    return model

In [17]:
from nlp.segment.model import build_glove_bi_lstm_crf_model
build_model = build_glove_bi_gru2_td_crf_model

In [18]:
def build_segment():
    model_name = 'seg-glove-bi-lstm-crf-model-mask'
    model_name = 'seg-glove-bi-gru2-td-crf-model-mask'

    args = ObjectDict()
    args.vocab_size = dataset.vocab_size
    args.num_labels = dataset.num_labels

    args.model_path = os.path.join(data_path, model_path, model_name + '.h5')
    args.glove_path = os.path.join(data_path, glove_path)

    crf_seg = CRFSegment(args, dataset, mode)
    return crf_seg

In [14]:
mode = 'train'

def train():    
    crf_seg = build_segment()

    train_data_path = os.path.join(data_path, corpus_path)
    train_x, train_y = dataset.process_input_data(train_data_path, maxlen=MAX_LENGTH)
    crf_seg.train(train_x, train_y)

    text_list = ['我是中国人',
                 '我爱北京天安门',
                 '郭小美和王帅身穿和服走在大街上',
                 '李冰冰从马上跳下来',
                 '武汉市长江大桥发表重要讲话',
                 '人们常说生活是一部教科书，而血与火的战争更是不可多得的教科书，她确实是名副其实的‘我的大学’。',
                 '为了有效地解决“高产穷县”的矛盾，吉林省委、省政府深入实际，调查研究，确定了实施“三大一强”的农业发展战略，即经过的努力，'
                 '粮食产量要再上两个台阶，畜牧业要成为农民增收的支柱产业，农副产品加工业要成为全省工业和财政收入的一大支柱，真正成为粮食"'
                 ]

    for text in text_list:
        print(crf_seg.seg(text))

In [ ]:
train()

In [21]:
from imp import reload
from nlp.segment import tools
from nlp.segment.tools import evaluation_seg
reload(tools)

mode = 'seg'

def evaluation():
    seg = build_segment()
    gold = data_path+"/nlp/corpus/icwb2-data/gold/msr_test_gold.utf8"
    tools.evaluation_seg(seg, gold)

In [20]:
evaluation()

100%|██████████| 3985/3985 [09:58<00:00,  6.66it/s]

 correct : 1597.000000  0.400753  P: 0.849083  R: 0.820235
